# Web Scraping - Cars

In [8]:
# Importing libraries

from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm_notebook
from math import ceil
from datetime import datetime
from pathlib import Path

In [9]:
def url_reader(url) -> object:

    """"
    This function reads the html code of the url and returns a BeautifulSoup object.

    Parameters:
    url (str): The url to be read.
    """

    # Need to set user agent to avoid 403 error
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read().decode('utf-8')
        soup_obj = BeautifulSoup(html, 'html.parser')
        return soup_obj

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
        

In [10]:
def list_of_ads(soup) -> list:
    
    """
    This function returns a list of all the ads in the page.

    Parameters:
    soup (object): The BeautifulSoup object of the page.

    Returns:
    ad_links (list): A list of all the ads in the page.
    """

    ad_list = soup.findAll('a', class_='fnmrjs-0 fyjObc')

    ad_links = []
    for ad in ad_list:
        ad_links.append((ad.get('href')))
        
    return ad_links

In [11]:
def text_from_tag(tag):

    """
    This function returns the text from a tag.

    Parameters:
    tag (object): The BeautifulSoup object of the page.

    Returns:
    text (str): The text from the tag.
    """

    if tag is not None:
        return tag.get_text()
    else:
        return 'None'

In [12]:
def main(get_images=False, number_of_pages=None, main_page=None):

    """
    This function scrapes the olx website and returns a dataframe with all the ads properties.

    Parameters:
    get_images (bool): If True, the images of the ads will be downloaded.
    number_of_pages (int): The number of pages to be scraped.

    Returns:
    df (pandas.DataFrame): A dataframe with all the ads properties.
    """

    # Reads the main page
    soup = url_reader(main_page)

    # Gets the number of pages and ads
    aux = soup.find('span', class_='sc-1mi5vq6-0 eDXljX sc-ifAKCX fhJlIo').get_text().split()
    ad_per_page = int(aux[2])
    n_total_ads = int(aux[-2].replace('.', ''))


    # If number_of_pages is not specified, calculate the number of pages
    if number_of_pages is not None:
        n_pages = number_of_pages
    else:
        n_pages = ceil(n_total_ads/ad_per_page)

    # Initialize the dataframe
    all_ad_data = []

    # Loop over the pages
    for page_number in tqdm_notebook(range(n_pages), desc='Pages', colour='red'):
        soup = url_reader(main_page + '?o={}'.format(page_number+1))
        ad_links = list_of_ads(soup)
        
        # Loop through all the ads
        for ad in ad_links:
            soup = url_reader(ad)
            if soup is not None:
            
                # Title
                title_tag = soup.find('h1', class_='sc-1q2spfr-0 lcTcEs sc-ifAKCX cmFKIN')
                title = text_from_tag(title_tag)

                # ID
                id_tag = soup.find('span', class_='sc-16iz3i7-0 qJvUT sc-ifAKCX fizSrB')
                id_ad = text_from_tag(id_tag).split()[-1]

                # Date and hour
                datetime_tag = soup.find('span', class_='sc-1oq8jzc-0 jvuXUB sc-ifAKCX fizSrB')
                datetime = text_from_tag(datetime_tag).split()
                date_ad, hour_ad = datetime[2], datetime[-1]

                # Description
                description_tag = soup.find('span', class_='sc-1sj3nln-1 eOSweo sc-ifAKCX cmFKIN')
                description = text_from_tag(description_tag).replace('\n',' ')

                # Price
                price_tag = soup.find('h2', class_='sc-ifAKCX eQLrcK')
                price = text_from_tag(price_tag).replace('.','')

                # Ad class
                calss_ad = 'amateur'
                if soup.find('span', class_='sc-16bj9n5-0 IIBHN sc-ifAKCX fizSrB') is not None: 
                    calss_ad = 'professional'

                # Vehicle data
                items_tags = soup.findAll('span', class_='sc-ifAKCX dCObfG')
                data = []
                if items_tags is not None:
                    items_tags.pop()
                    for tag in items_tags:
                        data.append(tag.get_text())

                # Vehicle subdata
                subitems_tags = soup.findAll('div', class_='duvuxf-0 h3us20-0 hCwZcX') + soup.findAll('div', class_='duvuxf-0 h3us20-0 kjKryV')
                subdata = []
                if subitems_tags is not None:
                    for tag in subitems_tags:
                        a = tag.find('span', class_='sc-ifAKCX cmFKIN')
                        b = tag.find('a')

                        if a is None:
                            subdata.append(b.get_text())
                        else:
                            subdata.append(a.get_text())

                # Optionals
                optionals_tags_parent = soup.find('div', class_='sc-bwzfXH h3us20-0 cNYGOs')
                optionals = []
                if optionals_tags_parent is not None:
                    optionals_tags = optionals_tags_parent.findAll('div', class_="duvuxf-0 h3us20-0 jyICCp")
                    for tag in optionals_tags:
                        optionals.append(tag.get_text())

                # Location
                location_tags = soup.find('div', class_='h3us20-6 govcZZ').findAll('dd', class_='sc-1f2ug0x-1 ljYeKO sc-ifAKCX kaNiaQ')
                location_titles = ['CEP', 'City', 'Neighborhood']
                location = []
                if location_tags is not None:
                    for tag in location_tags:
                        location.append(tag.get_text())

                # Get images
                if get_images:
                    img_tags = soup.find('div', class_='h3us20-6 fAprjt')
                    # img_list = soup.find('div', class_='h3us20-6 fAprjt').findAll('img', class_='image')
                    # for i, img in enumerate(img_list):
                    #     urlretrieve(img.get('src'), './output/img/' + id_ad + '_' + str(i) + '.jpg')

                    # Checks if exists images
                    if img_tags is not None:
                        # Creates the path directory
                        Path("./output/img").mkdir(parents=True, exist_ok=True)

                        img_list = img_tags.findAll('img', class_='image')
                        if img_list is not None:
                            for i, img in enumerate(img_list):
                                urlretrieve(img.get('src'), './output/img/' + id_ad + '_' + str(i) + '.jpg')

                # Create a dictionary with all the data
                ad_data = {}
                ad_data['Title'] = title
                ad_data['ID'] = id_ad
                ad_data['Description'] = description
                ad_data['Date'] = date_ad
                ad_data['Hour'] = hour_ad
                ad_data['Class'] = calss_ad
                ad_data['Price'] = price

                for i, _ in enumerate(range(len(data))):
                    ad_data[data[i]] = subdata[i]
                
                if soup.find('div', class_='sc-bwzfXH h3us20-0 cNYGOs') is not None:
                    for o in optionals:
                        ad_data[o] = 'yes'

                for i, _ in enumerate(range(len(location))):
                    ad_data[location_titles[i]] = location[i]              

                ad_data['Link'] = ad

                # Append the dictionary to the list
                all_ad_data.append(ad_data)

        else:
            continue

    return all_ad_data

In [13]:
def create_csv(data):

    """
    This function creates a csv file with all the ads properties.

    Parameters:
    data (list): A list with all the ads properties.

    Returns:
    None
    """

    dataset = pd.DataFrame(data)

    today = datetime.now().strftime('%Y-%m-%d')

    # Creates the path directory
    Path("./output/data").mkdir(parents=True, exist_ok=True)

    return dataset.to_csv('./output/data/' + today + '_dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [14]:
# if number_of_pages is not specified, scrape all the ads from all the pages
alldata = main(
    get_images = True, 
    number_of_pages=1, 
    main_page='https://al.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios'
    )
create_csv(alldata)

Pages:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
# creates a dataframe and shows the first 5 rows
dataset = pd.DataFrame(alldata)
dataset.head()

,Title,ID,Description,Date,Hour,Class,Price,Categoria,Modelo,Marca,...,Direção hidráulica,Alarme,Som,Sensor de ré,CEP,City,Neighborhood,Link,Câmera de ré,Blindado
0,Hb20 1.0 Confort Style 2016. Carro muito novo....,963034099,Hb20 1.0 Confort Style 2016. Carro muito novo....,03/12,07:58,professional,46300,"Carros, vans e utilitários",HYUNDAI HB20 COMFORT STYLE 1.0 TB FLEX 12V MEC.,HYUNDAI,...,yes,yes,yes,yes,57603298,Palmeira dos Índios,Juca Sampaio,https://al.olx.com.br/alagoas/autos-e-pecas/ca...,NaN,NaN
1,Fiat 500 Impecável!,934416873,Direção elétrica Vidro Trava Alarme Roda de li...,03/12,07:53,amateur,31900,"Carros, vans e utilitários",FIAT 500 CULT 1.4 FLEX 8V EVO MEC.,FIAT,...,yes,yes,yes,yes,57045000,Maceió,Barro Duro,https://al.olx.com.br/alagoas/autos-e-pecas/ca...,NaN,NaN
2,FIAT IDEA COMPLETO,964657957,"Vendo Fiat idea 2013, completo!",03/12,07:46,amateur,30000,"Carros, vans e utilitários",FIAT IDEA ATTRACTIVE 1.4 FIRE FLEX 8V 5P,FIAT,...,yes,yes,yes,yes,57035200,Maceió,Ponta Verde,https://al.olx.com.br/alagoas/autos-e-pecas/ca...,NaN,NaN
3,HILUX SRX 2017/2017 DIESEL AUTOMÁTICA,964063983,Carro extra Funciona tudo Muito nova Chave ...,03/12,07:44,professional,205000,"Carros, vans e utilitários",TOYOTA HILUX CD SRX 4X4 2.8 TDI 16V DIESEL AUT.,TOYOTA,...,yes,yes,yes,yes,57100000,Rio Largo,NaN,https://al.olx.com.br/alagoas/autos-e-pecas/ca...,yes,NaN
4,Vende-se urgente,964656792,Chevette 91 tem que ajeitar o alternador bater...,03/12,07:37,amateur,1800,"Carros, vans e utilitários",GM - CHEVROLET CHEVETTE L / SL / SL/E / DL / S...,GM - CHEVROLET,...,NaN,NaN,NaN,NaN,57100000,Rio Largo,NaN,https://al.olx.com.br/alagoas/autos-e-pecas/ca...,NaN,NaN
